In [61]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, Dataset
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
print("Device:", device)

Device: cpu


In [12]:
with open('/content/drive/MyDrive/train.pkl', 'rb') as f:
    data = pickle.load(f)

In [54]:
data[0]

(array([ -1.,  -1.,  -1., ...,  78.,  40., 144.]), 0)

In [57]:
for i in data:
  for j in i[0]:
    if (j==-100):
      print(j)

In [62]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [58]:
pad = -100

def pad_collate(batch, pad_value=pad):
    xx, yy = zip(*batch)
    xx = [torch.tensor(x) for x in xx]
    x_lens = [len(x) for x in xx]
    yy_pad = torch.tensor(yy)
    y_lens = [1] * len(yy)
    # print(xx)
    # print(yy)

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    # yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [59]:
dataset_size = len(data)

train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_set, test_set = random_split(data, [train_size, test_size])

batch_size=4
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [21]:
print(len(train_set[0][0]))
print(len(train_set[1][0]))

216
264


In [60]:
next(iter(trainloader))

(tensor([[   0.,    0.,   13.,  ..., -100., -100., -100.],
         [  68.,   20.,  148.,  ...,    5.,  145.,   -1.],
         [  -1.,   -1.,   -1.,  ..., -100., -100., -100.],
         [  -1.,   -1.,   67.,  ..., -100., -100., -100.]], dtype=torch.float64),
 tensor([0, 0, 3, 0]),
 [64, 858, 232, 486],
 [1, 1, 1, 1])